In [2]:
import gradio as gr
import os
from IPython.display import Markdown
from python_scripts import llm_rag, machine_translation, text_to_speech, whisper_setup, get_audio, utils

# Define paths dynamically
PATH = os.getcwd().replace('\\\\', '/')
data_path = PATH + '/Datasets/MeDAL'

In [3]:
def SMTS():
    try:
        # Step 1: Get Query from Audio
        audio_path = utils.record_audio()

        # Step 2: Transcribe Query to English
        whisper_models = ["tiny", "base", "small", "medium", "large"]
        transcript = whisper_setup.transcribe_audio(audio_path, whisper_models)

        # Step 3: Feed query into the LLM
        models = { 
            'llama_13b': 'a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5',
            'mixtral': 'mistralai/mixtral-8x7b-instruct-v0.1',
            'llama_70b': 'meta/llama-2-70b-chat:2796ee9483c3fd7aa2e171d38f4ca12251a30609463dcfd4cd76703f22e96cdf'
        }

        model = llm_rag.DocumentEmbeddingPipeline(model_version = models['mixtral'])
        model.setup_environment()
        model.prepare_documents(data_path = data_path, collection_name = "ubaid_notes_2", joining = True, persistent = False)
        model.embed_and_index()

        response = [model.query_data(query = transcript[i]['text']) for i in range(len(whisper_models))]

        # Step 4: Translate it back to the user language
        translated_text = machine_translation.translate_text(text=" ".join(response), src_lang = 'en', trg_lang=transcript[0]['lang'])

        # Step 5: Now speak the response in the user's language
        answer_path = utils.generate_answer_path(original_path=audio_path)
        text_to_speech.multilingual_text_to_speech(text=translated_text, filepath=answer_path)

        display(Markdown(f"<b>{translated_text}</b>"))
    except Exception as e:
        print("An error occurred:", e)

In [4]:
iface = gr.Interface(
    fn=SMTS,
    inputs="button",  
    outputs="text"
)

In [5]:
if __name__ == "__main__":
    iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\queueing.py", line 527, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\route_utils.py", line 261, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1788, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\blocks.py", line 1340, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Talha\AppData\Local\Programs\Python\Python311\Lib\sit